# 0.0. __INIT__

## 0.1. Imports

## 0.2. Helper Functions

## 0.3. Loading data

# 1.0. DATA DESCRIPTION

## 1.1. Rename Columns

## 1.2. Data Dimensions

## 1.3. Data Types

## 1.4. Check NA

## 1.5. Fillout NA

## 1.6. Change Types

## 1.7. Descriptive Statistical

### 1.7.1. Numerical Attributes

### 1.7.2. Categorical Attrributes

# 2.0. FEATURE ENGENEERING

## 2.1. Hypothesis creation

### 2.1.1. Store Hypothesis

## 2.3. Featuring engineering

# 3.0. DATA FILTERING

## 3.1. Lines filtering

## 3.2. Columns filtering

# 4.0. EXPLORATORY DATA ANALISYS (EDA)

## 4.1. Univariate Analysis

### 4.1.1. Response Variable

### 4.1.2. Numerical Variables

### 4.1.3. Categorical Variables

## 4.2. Bivariate Analysis

### **H1.** Lojas com maior sortimento deveriam vender mais.
**FALSA** Lojas com MAIOR SORTIMENTO vendem MENOS

### **H2.** Lojas com competidores mais perto deveriam vender mais.
**VERDADEIRA**

### **H3.** Lojas com competidores há mais tempo deveriam vender mais.
**FALSA** Lojas com competidores há MAIS TEMPO vendem MENOS

### **H4.** Lojas com promoções ativas por mais tempo deveriam vender mais.
**FALSA** Lojas com promoções ativas por MAIS TEMPO vendem MENOS depois de um certo período de tempo

### 4.2.1. Hypothesis summary

## 4.3. Multivariate Analysis

### 4.3.1 Numerical Attributes

### 4.3.2 Categorical Attributes

# 5.0. DATA PREPARATION

## 5.1. Normalization
Because it was not possible to identify any normal distribution in section 4.1.2., there will be no data normalization.

## 5.2. Rescaling

## 5.3. Transformation

### 5.3.1. Encoding

### 5.3.2. Response variable transformation

### 5.3.3. Nature transformation

# 6.0. FEATURE SELECTION

## 6.1. Split dataframe into training and test datasets

## 6.2. Feature Selector

### 6.2.1. Best Features

## 6.3. Feature Selection

# 7.0. MACHINE LEARNING MODELLING

## 7.1. Baseline Model

## 7.2. Linear Regression Model

### 7.2.1. Linear Regression Model - Cross Validation

## 7.3. Linear Regression Regularized Model - Lasso

### 7.3.1. Linear Regression Regularized Model - Cross Validation

## 7.4. Random Forest Regressor

### 7.4.1. Random Forest Regressor - Cross Validation

## 7.5. XGBoost Regressor

### 7.5.1. XGBoost Regressor - Cross Validation

## 7.6. Model comparison

### 7.6.1. Single Performance

### 7.6.2. Real Performance - Cross Validation

# 8.0. HYPERPARAMETER FINE TUNING


## 8.1. Random Search

## 8.2. Final Model

# 9.0. ERROR TRANSLATION AND INTERPRETATION


## 9.1. Business Performance

## 9.2. Total Performance

## 9.3. Machine Learning Performance

# 10.0. DEPLOY MODEL TO PRODUCTION


## 10.1. Rossmann Class

## 10.2. API Handler

## 10.3. API Tester